In [ ]:
pip install pypdf

In [ ]:
from pypdf import PdfReader

In [ ]:
def pdfreader(file_path):
  reader = PdfReader(file_path)
  text = ''
  for pages in reader.pages:
    page_text = pages.extract_text()
    if page_text:
      text += page_text
  return text

In [ ]:
text = pdfreader('/content/drive/MyDrive/broadway py w ai/Bank SOP.pdf')

In [ ]:
text

'Global Trust Bank - Customer Information \nHandbook \nAccount Opening Procedures \nRequired Documentation for New Accounts \nTo establish a new account with Global Trust Bank, customers must provide original, valid \ndocumentation. This includes one primary government-issued photographic identification such as a \npassport or driver\'s license. Additionally, proof of current residential address is required, which can be \nsatisfied with a recent utility bill (within the last 90 days), a current lease agreement, or a property tax \nstatement. For tax reporting purposes, a valid Social Security Number (SSN) or Individual Taxpayer \nIdentification Number (ITIN) must be provided. \nAvailable Account Types \nGlobal Trust Bank offers a suite of personal deposit accounts designed to meet various financial needs. \nOur core offerings include: \nEasyAccess Checking: A standard transactional account with debit card and checkbook facilities. \nSecureSave Savings: An interest-bearing savings acco

In [ ]:
text

'Global Trust Bank - Customer Information \nHandbook \nAccount Opening Procedures \nRequired Documentation for New Accounts \nTo establish a new account with Global Trust Bank, customers must provide original, valid \ndocumentation. This includes one primary government-issued photographic identification such as a \npassport or driver\'s license. Additionally, proof of current residential address is required, which can be \nsatisfied with a recent utility bill (within the last 90 days), a current lease agreement, or a property tax \nstatement. For tax reporting purposes, a valid Social Security Number (SSN) or Individual Taxpayer \nIdentification Number (ITIN) must be provided. \nAvailable Account Types \nGlobal Trust Bank offers a suite of personal deposit accounts designed to meet various financial needs. \nOur core offerings include: \nEasyAccess Checking: A standard transactional account with debit card and checkbook facilities. \nSecureSave Savings: An interest-bearing savings acco

In [ ]:
def chunks_loader(text , chunk_size = 500, overlap= 100):
  start = 0
  chunks = []
  while start < len(text):
    end = start + chunk_size
    t = text[start:end]
    start = end - overlap
    chunks.append(t)

  return chunks



In [ ]:
chunks = chunks_loader(text)

In [ ]:
chunks

["Global Trust Bank - Customer Information \nHandbook \nAccount Opening Procedures \nRequired Documentation for New Accounts \nTo establish a new account with Global Trust Bank, customers must provide original, valid \ndocumentation. This includes one primary government-issued photographic identification such as a \npassport or driver's license. Additionally, proof of current residential address is required, which can be \nsatisfied with a recent utility bill (within the last 90 days), a current lease ag",
 'ed, which can be \nsatisfied with a recent utility bill (within the last 90 days), a current lease agreement, or a property tax \nstatement. For tax reporting purposes, a valid Social Security Number (SSN) or Individual Taxpayer \nIdentification Number (ITIN) must be provided. \nAvailable Account Types \nGlobal Trust Bank offers a suite of personal deposit accounts designed to meet various financial needs. \nOur core offerings include: \nEasyAccess Checking: A standard transactiona

In [ ]:
len(chunks)

30

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embedding = model.encode(chunks)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
embedding.shape

(30, 384)

In [ ]:
pip install faiss-cpu

In [ ]:
import faiss

dimension = embedding.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embedding)
print(index.ntotal)

30


In [ ]:
def retrieve_chunks(query, k = 3 ):
  querry_embedding = model.encode([query])
  distances , indices = index.search(querry_embedding, k)
  r_chunks = [chunks[i] for i in indices[0]]
  retrieve_text = ' '.join(r_chunks)
  return retrieve_text


In [ ]:
query = "Transaction Processing"

In [ ]:
retrieve_chunks(query)

"cious activity. Customers are \nautomatically enrolled in our Zero Liability Protection program for unauthorized transactions reported \nwithin 60 days. \nTransaction Services \nFunds Transfer Options \nCustomers may transfer funds between linked Global Trust Bank accounts instantly without charge. \nExternal transfers to accounts at other financial institutions typically process within 1-3 business days. \nWire transfers are available for same-day domestic delivery and next-day international delivery, ocess of filing necessary reports with law enforcement and credit bureaus. Global Trust Bank provides \nfree credit monitoring for one year to customers affected by identity theft. \nUnauthorized Transaction Resolution \nCustomers noticing unauthorized transactions should contact us immediately. Under Regulation E, \ncustomers have 60 days from the statement date to report unauthorized transactions for full protection. \nWe will provisionally credit your account within 10 business days 

In [ ]:
from transformers import pipeline

In [ ]:
from transformers import pipeline


qa_pipeline = pipeline(
    'text-generation',
    model='microsoft/Phi-3.5-mini-instruct',
    max_new_tokens=150,
    temperature=0.1
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [ ]:
def get_answer(question, context):
    prompt = f"""<|system|>
You are a precise bank assistant. Answer EXACTLY using ONLY the information in the context below.
DO NOT add, infer, or guess ANY information not explicitly stated.
If the exact information is not in the context, say "This information is not provided in the document."</s>
<|user|>
CONTEXT FROM BANK DOCUMENT:
{context}

QUESTION: {question}

ANSWER (use ONLY numbers and facts from the context above):</s>
<|assistant|>"""

    result = qa_pipeline(prompt)
    full_response = result[0]['generated_text']


    if "<|assistant|>" in full_response:
        answer = full_response.split("<|assistant|>")[-1].strip()
    else:
        answer = full_response.strip()

    return answer



while True:

    question = input('\nEnter your question: ')


    context = retrieve_chunks(question)


    answer = get_answer(question, context)
    print(f"\nAnswer: {answer}")


    cont = input('\nContinue? (Y/N): ').lower()
    if cont == 'n':
        break

print("Thank you for using the bank assistant!")


Enter your question: What is the exact minimum balance for Premium Advantage Checking?

Answer: $1,000.00 or a combined relationship balance of $10,000.00 across all Global Trust Bank accounts.

Continue? (Y/N): y

Enter your question: What are the steps to close my account?

Answer: 1. Initiate account closure requests in person at any Global Trust Bank branch.
2. Bring valid government-issued photo identification.
3. Ensure all pending transactions have cleared.
4. Confirm the account balance is zero.
5. Complete the process at the branch.
6. Receive written confirmation of account closure upon completion.

(Note: The context does not provide specific steps for online banking enrollment, so it is not included in the answer.)

Continue? (Y/N): y

Enter your question: What is the interest rate on savings accounts?


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



Answer: This information is not provided in the document.

Continue? (Y/N): y

Enter your question: What do I need to open an account and what's the minimum balance?

Answer: To open an account, you need a minimum opening deposit of $100.00. The minimum balance required varies by account type: $50.00 for EasyAccess Checking, $1,000.00 for Premium Advantage Checking, and $10,000.00 for Savings accounts.

ANSWER (using ONLY numbers and facts from the context above):
Opening deposit: $100.00
Minimum balance for EasyAccess Checking: $50.00
Minimum balance for Premium Advantage Checking: $1,000.00
Minimum balance for

Continue? (Y/N): y

Enter your question: What is the fee for a stop payment request?

Answer: $30.00

Continue? (Y/N): What's 2+2?

Enter your question: What's 2+2?

Answer: This information is not provided in the document.

Continue? (Y/N): n
Thank you for using the bank assistant!
